### Find Word Group Youtube Link

In [101]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from nltk import ngrams
from functools import reduce
from pathlib import Path
import shutil

In [102]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [103]:
# language pair
lang_folder = "Dutch"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 200  # 28 native word end index

# youtube
sample_num = 7  # 7
time_shift = 0.6

In [104]:
def df_col_value_join_comma(df, df_columns_list):
    '''
    df_col_value_join_comma(df_test, ["video_id","start_time","end_time"])\n
    function used for selected column value join with comma in one row
    '''
    column_value_list = []
    for column in df_columns_list:
        list_var = df[f"{column}"].to_list()
        list_var_string = [str(x) for x in list_var] 
        list_var_join = ",".join(list_var_string)
        column_value_list.append(list_var_join)

    df_result = pd.DataFrame([[column_value_list[0],column_value_list[1],column_value_list[2]]], columns=df_columns_list)

    return df_result

In [105]:
def word_in_wordgroup_simple(source_word_list, df_target, target_column, word_sample_num, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column row result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,"search_string",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [106]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)\n
    df_youtube_sentence is dataframe and "sentence" is its column for external searching_list
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].sample(sample_num)
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [107]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")\n
    df_search_result is dataframe and "search_string", "start_time", "end_time", "sentence", "video_id" are its columns
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [108]:
def word_count_result(df, column_list, set_condition=False): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        if set_condition:
            var_list = set(df[f"{i}"].dropna().tolist())
        else:
            var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    df_word_count.sort_values("word_count", ascending=False, inplace=True)
    df_word_count.reset_index(inplace=True, drop=True)
    
    return  df_word_count

In [109]:
def word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max):
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_result.sort_values(by="frequency", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [110]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Result/Adjust Data Repeat"

Path(path).mkdir(parents=True, exist_ok=True)

In [111]:
#disable_video_id_list = ["H6E6N70jYqI","BoaYsdPtJYA","HUZINsU40Fk","Et3diPcEmfY"]
disable_video_id_list = []

In [112]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,ik,22539531
1,je,20769946
2,het,15696161
3,de,15258816
4,dat,13387137
...,...,...
331769,oorlogsgezichten,5
331770,opdrachtenlijst,5
331771,verlsaafde,5
331772,oxidatieproces,5


In [113]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [114]:
df_word_select

,word,frequency
0,ik,22539531
1,je,20769946
2,het,15696161
3,de,15258816
4,dat,13387137
...,...,...
195,genoeg,353128
196,wist,352948
197,oh,351988
198,klaar,350506


In [115]:
word_list = df_word_select["word"].to_list()
len(word_list)

200

In [116]:
if twogram_sentence_check:
    df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_twogram = df_twogram_sent.loc[:,["twogram","frequency"]]
else:
    df_twogram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Twogram_Merge.csv")  
    df_twogram = df_twogram.loc[:,["twogram","frequency"]]

df_twogram

,twogram,frequency
0,kom op,184578
1,dank je,127178
2,dank u,64014
3,wacht even,63826
4,echt waar,59238
...,...,...
328541,zesentwintig horizontaal,3
328542,volledig gedisciplineerd,3
328543,volledig geconcentreerd,3
328544,hero netegengif,3


In [117]:
# Twogram Func
d_list2  = df_twogram.iloc[:,0].values.tolist()

resultlist2 = []
manager = multiprocessing.Manager()
resultlist2 = manager.list()

def word_in_wordgroup(d_list2):
    mergelist2 = []
    try:
        word = d_list2.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist2.append(word[j])
            if len(mergelist2) == len(word):
                    resultlist2.append(d_list2)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list2) # string_word liste

result_list2 = list(resultlist2)
df_result2 = pd.DataFrame(result_list2, columns=[0])  # add columns parameter for empty result
df_result2 = df_result2.rename(columns = {0: "twogram"})
df_merge2 = pd.merge(df_result2, df_twogram, how="left", on="twogram")
df_merge_result2 = df_merge2.sort_values(by="frequency", ascending=False)
df_merge_result2.drop_duplicates(inplace=True)
df_merge_result2.reset_index(drop=True, inplace=True)
df_twogram_select = df_merge_result2
df_twogram_select

,twogram,frequency
0,kom op,184578
1,dank je,127178
2,dank u,64014
3,wacht even,63826
4,echt waar,59238
...,...,...
6564,dat iemand,3
6565,allemaal toch,3
6566,meer om,3
6567,die helpen,3


In [118]:
df_twogram_word_usage = word_usage_result(word_list, df_twogram_select, "twogram", "frequency", min_word_use_num, max_word_use_num)
df_twogram_word_usage

,twogram,frequency
0,kom op,184578
1,dank je,127178
2,dank u,64014
3,wacht even,63826
4,echt waar,59238
...,...,...
577,bij zich,3
578,tot tot,3
579,nodig bent,3
580,gedaan heb,3


In [119]:
if threegram_sentence_check:
    df_threegram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Three_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_threegram_sent.rename(columns={"three_gram":"threegram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_threegram = df_threegram_sent.loc[:,["threegram","frequency"]]
else:
    df_threegram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Threegram_Merge.csv")  
    df_threegram = df_threegram.loc[:,["threegram","frequency"]]

df_threegram

,threegram,frequency
0,het spijt me,132758
1,wat is er,106497
2,ik weet het,67518
3,wat is dat,62768
4,wat doe je,60323
...,...,...
550436,vliegen onbekend aantal,3
550437,vliegen obstakel afnemen,3
550438,voor achtduizend euro,3
550439,voor de kijkcijfers,3


In [120]:
# Threegram Func
d_list3  = df_threegram.iloc[:,0].values.tolist()

resultlist3 = []
manager = multiprocessing.Manager()
resultlist3 = manager.list()

def word_in_wordgroup(d_list3):
    mergelist3 = []
    try:
        word = d_list3.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist3.append(word[j])
            if len(mergelist3) == len(word):
                    resultlist3.append(d_list3)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list3) # string_word liste

result_list3 = list(resultlist3)
df_result3 = pd.DataFrame(result_list3, columns=[0])  # add columns parameter for empty result
df_result3 = df_result3.rename(columns = {0: "threegram"})
df_merge3 = pd.merge(df_result3, df_threegram, how="left", on="threegram")
df_merge_result3 = df_merge3.sort_values(by="frequency", ascending=False)
df_merge_result3.drop_duplicates(inplace=True)
df_merge_result3.reset_index(drop=True, inplace=True)
df_threegram_select = df_merge_result3
df_threegram_select

,threegram,frequency
0,het spijt me,132758
1,wat is er,106497
2,ik weet het,67518
3,wat is dat,62768
4,wat doe je,60323
...,...,...
29304,wat doen dan,3
29305,omdat wij weten,3
29306,waar ze staat,3
29307,iedereen bij deze,3


In [121]:
df_threegram_word_usage = word_usage_result(word_list, df_threegram_select, "threegram", "frequency", min_word_use_num, max_word_use_num)
df_threegram_word_usage = df_threegram_word_usage.head(50)  # ????
df_threegram_word_usage

,threegram,frequency
0,het spijt me,132758
1,wat is er,106497
2,ik weet het,67518
3,wat is dat,62768
4,wat doe je,60323
5,wat is dit,45582
6,dat weet ik,39006
7,hoe gaat het,35978
8,weet je wat,31426
9,dank je wel,30479


In [122]:
#df_threegram_word_usage.to_excel("Dutch_Threegram.xlsx", index=False)

In [123]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:10.760,00:00:13.200,jongens hier ben ik dan,WA8MV8nX9s0
1,00:00:13.200,00:00:17.040,en we gaan iets heel maar dan ook heel geks me...,WA8MV8nX9s0
2,00:00:17.040,00:00:19.320,ik bedoel we gaan naar de royal bridge suite,WA8MV8nX9s0
3,00:00:19.900,00:00:23.760,dit is de meest luxe suite die je maar kan bed...,WA8MV8nX9s0
4,00:00:23.760,00:00:25.860,de meest bekende mensen hebben hier geslapen,WA8MV8nX9s0
...,...,...,...,...
611565,00:01:10.600,00:01:12.440,ik geloof in de kracht van kinderen,N0rCE0Ot8Os
611566,00:01:12.760,00:01:16.160,die instelling geven wij ook mee geloof in jezelf,N0rCE0Ot8Os
611567,00:01:17.000,00:01:18.920,mijn naam is melvin kolf,N0rCE0Ot8Os
611568,00:01:19.000,00:01:22.160,wil jij meer weten over leren en werken in dez...,N0rCE0Ot8Os


In [124]:
df_youtube_sentence = df_youtube_sentence[~df_youtube_sentence["video_id"].isin(disable_video_id_list)]
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:10.760,00:00:13.200,jongens hier ben ik dan,WA8MV8nX9s0
1,00:00:13.200,00:00:17.040,en we gaan iets heel maar dan ook heel geks me...,WA8MV8nX9s0
2,00:00:17.040,00:00:19.320,ik bedoel we gaan naar de royal bridge suite,WA8MV8nX9s0
3,00:00:19.900,00:00:23.760,dit is de meest luxe suite die je maar kan bed...,WA8MV8nX9s0
4,00:00:23.760,00:00:25.860,de meest bekende mensen hebben hier geslapen,WA8MV8nX9s0
...,...,...,...,...
611565,00:01:10.600,00:01:12.440,ik geloof in de kracht van kinderen,N0rCE0Ot8Os
611566,00:01:12.760,00:01:16.160,die instelling geven wij ook mee geloof in jezelf,N0rCE0Ot8Os
611567,00:01:17.000,00:01:18.920,mijn naam is melvin kolf,N0rCE0Ot8Os
611568,00:01:19.000,00:01:22.160,wil jij meer weten over leren en werken in dez...,N0rCE0Ot8Os


In [125]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

<ipython-input-125-a8911ca4399e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second
<ipython-input-125-a8911ca4399e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])


In [126]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

<ipython-input-126-aa25c83efaa1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
<ipython-input-126-aa25c83efaa1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())


,start_time,end_time,sentence,video_id
0,10.76,13.20,jongens hier ben ik dan,WA8MV8nX9s0
1,13.20,17.04,en we gaan iets heel maar dan ook heel geks me...,WA8MV8nX9s0
2,17.04,19.32,ik bedoel we gaan naar de royal bridge suite,WA8MV8nX9s0
3,19.90,23.76,dit is de meest luxe suite die je maar kan bed...,WA8MV8nX9s0
4,23.76,25.86,de meest bekende mensen hebben hier geslapen,WA8MV8nX9s0
...,...,...,...,...
611565,70.60,72.44,ik geloof in de kracht van kinderen,N0rCE0Ot8Os
611566,72.76,76.16,die instelling geven wij ook mee geloof in jezelf,N0rCE0Ot8Os
611567,77.00,78.92,mijn naam is melvin kolf,N0rCE0Ot8Os
611568,79.00,82.16,wil jij meer weten over leren en werken in dez...,N0rCE0Ot8Os


In [142]:
#search_list = df_twogram_word_usage["twogram"].to_list()
search_list = df_threegram_word_usage["threegram"].iloc[10:19,].to_list()

In [143]:
len(search_list)

9

In [144]:
df_word_group = word_group_youtube(df_youtube_sentence, search_list, "sentence", sample_num)
df_word_group

,search_string,start_time,end_time,sentence,video_id
0,is dat zo,599.640,603.920,en op moment dat ik het een keer verkeerd loop...,VjYg2X32ryM
1,is dat zo,91.542,96.876,wat is er aan de hand in iran en waarom is dat...,ypikak3H9MI
2,is dat zo,623.281,625.176,is dat zo scherp bij transport van mest,USIYNJg2qSw
3,is dat zo,17.799,24.190,is dat zo ja ik werd vanochtend wakker om zes ...,NMUAd5R67K0
4,is dat zo,73.160,75.160,is dat zo zijn jullie altijd zo rustig,cUB3dxLXvSc
...,...,...,...,...,...
58,weet ik niet,41.420,43.780,hoe het eruit ziet als het straks gemengd is d...,RR2Xly7O0jA
59,weet ik niet,3788.080,3789.800,dat weet ik niet,1idPrF17UP0
60,weet ik niet,1380.570,1385.669,dat klopt dat is het en dan krijg je dat ie ee...,XRBb17aWENc
61,weet ik niet,158.080,160.080,weet ik niet ik keek hier door heen,SY4Gv78Mxbw


In [145]:
df_word_group_time_loc = word_group_time_loc(df_word_group, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc

,search_string,start_time,end_time,sentence,video_id
0,is dat zo,603.251250,603.920000,en op moment dat ik het een keer verkeerd loop...,VjYg2X32ryM
1,is dat zo,95.324291,96.391091,wat is er aan de hand in iran en waarom is dat...,ypikak3H9MI
2,is dat zo,623.281000,623.766897,is dat zo scherp bij transport van mest,USIYNJg2qSw
3,is dat zo,17.799000,18.639921,is dat zo ja ik werd vanochtend wakker om zes ...,NMUAd5R67K0
4,is dat zo,73.160000,73.686316,is dat zo zijn jullie altijd zo rustig,cUB3dxLXvSc
...,...,...,...,...,...
58,weet ik niet,43.277049,43.780000,hoe het eruit ziet als het straks gemengd is d...,RR2Xly7O0jA
59,weet ik niet,3788.402500,3789.800000,dat weet ik niet,1idPrF17UP0
60,weet ik niet,1384.683966,1385.495170,dat klopt dat is het en dan krijg je dat ie ee...,XRBb17aWENc
61,weet ik niet,158.080000,158.822857,weet ik niet ik keek hier door heen,SY4Gv78Mxbw


In [146]:
df_word_group_time_loc.start_time = df_word_group_time_loc.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc.end_time = df_word_group_time_loc.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc

,search_string,start_time,end_time,sentence,video_id
0,is dat zo,602.651250,604.520000,en op moment dat ik het een keer verkeerd loop...,VjYg2X32ryM
1,is dat zo,94.724291,96.991091,wat is er aan de hand in iran en waarom is dat...,ypikak3H9MI
2,is dat zo,622.681000,624.366897,is dat zo scherp bij transport van mest,USIYNJg2qSw
3,is dat zo,17.199000,19.239921,is dat zo ja ik werd vanochtend wakker om zes ...,NMUAd5R67K0
4,is dat zo,72.560000,74.286316,is dat zo zijn jullie altijd zo rustig,cUB3dxLXvSc
...,...,...,...,...,...
58,weet ik niet,42.677049,44.380000,hoe het eruit ziet als het straks gemengd is d...,RR2Xly7O0jA
59,weet ik niet,3787.802500,3790.400000,dat weet ik niet,1idPrF17UP0
60,weet ik niet,1384.083966,1386.095170,dat klopt dat is het en dan krijg je dat ie ee...,XRBb17aWENc
61,weet ik niet,157.480000,159.422857,weet ik niet ik keek hier door heen,SY4Gv78Mxbw


In [147]:
df_word_group_time_loc.start_time = df_word_group_time_loc.start_time.apply(lambda x: round(x))
df_word_group_time_loc.end_time = df_word_group_time_loc.end_time.apply(lambda x: round(x))
df_word_group_time_loc 

,search_string,start_time,end_time,sentence,video_id
0,is dat zo,603,605,en op moment dat ik het een keer verkeerd loop...,VjYg2X32ryM
1,is dat zo,95,97,wat is er aan de hand in iran en waarom is dat...,ypikak3H9MI
2,is dat zo,623,624,is dat zo scherp bij transport van mest,USIYNJg2qSw
3,is dat zo,17,19,is dat zo ja ik werd vanochtend wakker om zes ...,NMUAd5R67K0
4,is dat zo,73,74,is dat zo zijn jullie altijd zo rustig,cUB3dxLXvSc
...,...,...,...,...,...
58,weet ik niet,43,44,hoe het eruit ziet als het straks gemengd is d...,RR2Xly7O0jA
59,weet ik niet,3788,3790,dat weet ik niet,1idPrF17UP0
60,weet ik niet,1384,1386,dat klopt dat is het en dan krijg je dat ie ee...,XRBb17aWENc
61,weet ik niet,157,159,weet ik niet ik keek hier door heen,SY4Gv78Mxbw


In [148]:
df_word_group_time_loc["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc['video_id'].map(str)+"&t="+df_word_group_time_loc['start_time'].map(str)+"s"
df_word_group_time_loc

,search_string,start_time,end_time,sentence,video_id,video_url
0,is dat zo,603,605,en op moment dat ik het een keer verkeerd loop...,VjYg2X32ryM,https://www.youtube.com/watch?v=VjYg2X32ryM&t=...
1,is dat zo,95,97,wat is er aan de hand in iran en waarom is dat...,ypikak3H9MI,https://www.youtube.com/watch?v=ypikak3H9MI&t=95s
2,is dat zo,623,624,is dat zo scherp bij transport van mest,USIYNJg2qSw,https://www.youtube.com/watch?v=USIYNJg2qSw&t=...
3,is dat zo,17,19,is dat zo ja ik werd vanochtend wakker om zes ...,NMUAd5R67K0,https://www.youtube.com/watch?v=NMUAd5R67K0&t=17s
4,is dat zo,73,74,is dat zo zijn jullie altijd zo rustig,cUB3dxLXvSc,https://www.youtube.com/watch?v=cUB3dxLXvSc&t=73s
...,...,...,...,...,...,...
58,weet ik niet,43,44,hoe het eruit ziet als het straks gemengd is d...,RR2Xly7O0jA,https://www.youtube.com/watch?v=RR2Xly7O0jA&t=43s
59,weet ik niet,3788,3790,dat weet ik niet,1idPrF17UP0,https://www.youtube.com/watch?v=1idPrF17UP0&t=...
60,weet ik niet,1384,1386,dat klopt dat is het en dan krijg je dat ie ee...,XRBb17aWENc,https://www.youtube.com/watch?v=XRBb17aWENc&t=...
61,weet ik niet,157,159,weet ik niet ik keek hier door heen,SY4Gv78Mxbw,https://www.youtube.com/watch?v=SY4Gv78Mxbw&t=...


In [149]:
word_count_result(df_word_group_time_loc, ["search_string"], set_condition=True)

,word,word_count
0,is,4
1,wie,2
2,ik,2
3,dat,2
4,gaan,2
5,laten,1
6,goed,1
7,het,1
8,hij,1
9,je,1


In [150]:
#df_word_group_time_loc.to_excel(f"{lang_folder.capitalize()}_Word_Group_With_{word_end}_Word_{sample_num}_Youtube_{time_shift}s_Timeshift_For_Talk_Time_Result.xlsx", index=False) 